In [61]:
%pip install numpy
%pip install pandas
%pip install sklearn
import numpy as np
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
from sklearn.impute import KNNImputer
from sklearn.cluster import DBSCAN

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
df = pd.read_csv("water_potability.csv")
desc = df.describe()
desc

In [ ]:
df_fringeless = df.copy(deep=True)
for col in df.columns:
    mean = desc[col]['mean']
    std = desc[col]['std']
    tmp = df[df[col] > mean + std*3]
    tmp = tmp.append(df[df[col] < mean - std*3])
    df_fringeless = df_fringeless.drop(tmp.index, errors='ignore')
    display(f"|{col}|mean:{mean}|std:{std}|")
    display(tmp)

In [ ]:
display(len(df) - len(df_fringeless))
display(df_fringeless.describe())

In [ ]:
imputer = KNNImputer(n_neighbors=3)
df_filled = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [80]:

clf = LocalOutlierFactor(n_neighbors=3)
outliers = clf.fit_predict(df_filled)
outliers_indexes = np.where(outliers == - 1)[0]
rest_indexes = np.where(outliers == 1)[0]
outliers = df_filled.loc[outliers_indexes]
df_without_outliers = df_filled.loc[rest_indexes]
df_without_outliers = df_without_outliers.reset_index().drop(columns=['index'])
print(df_without_outliers.describe())
print(outliers)

                ph     Hardness        Solids  Chloramines      Sulfate  \
count  3144.000000  3144.000000   3144.000000  3144.000000  3144.000000   
mean      7.074319   196.679788  21986.878217     7.126021   332.953221   
std       1.495752    31.611776   8734.131316     1.574584    36.605135   
min       0.000000    77.459586    320.942611     0.352000   129.000000   
25%       6.150537   177.504126  15626.864313     6.137411   311.669230   
50%       7.048972   197.251672  20990.355544     7.128299   333.057215   
75%       7.974111   216.493143  27313.401150     8.109320   355.176514   
max      14.000000   323.124000  56867.859236    13.127000   476.539717   

       Conductivity  Organic_carbon  Trihalomethanes    Turbidity   Potability  
count   3144.000000     3144.000000      3144.000000  3144.000000  3144.000000  
mean     425.804435       14.295640        66.348270     3.965209     0.387087  
std       78.630292        3.301064        15.834972     0.778282     0.487161  


In [81]:
print(df_without_outliers)

            ph    Hardness        Solids  Chloramines     Sulfate  \
0     6.655223  204.890455  20791.318981     7.300212  368.516441   
1     3.716080  129.422921  18630.057858     6.635246  351.285226   
2     8.099124  224.236259  19909.541732     9.275884  347.323743   
3     8.316766  214.373394  22018.417441     8.059332  356.886136   
4     9.092223  181.101509  17978.986339     6.546600  310.135738   
...        ...         ...           ...          ...         ...   
3139  4.668102  193.681735  47580.991603     7.166639  359.948574   
3140  7.808856  193.553212  17329.802160     8.061362  368.086095   
3141  9.419510  175.762646  33155.578218     7.350233  316.571962   
3142  5.126763  230.603758  11983.869376     6.303357  334.293598   
3143  7.874671  195.102299  17404.177061     7.509306  333.330087   

      Conductivity  Organic_carbon  Trihalomethanes  Turbidity  Potability  
0       564.308654       10.379783        86.990970   2.963135         0.0  
1       592.88535

In [127]:
normalized_df=(df_without_outliers-df_without_outliers.min())/(df_without_outliers.max()-df_without_outliers.min())
clustering = DBSCAN(eps=0.75).fit(normalized_df)
for label in range(clustering.n_features_in_):
    tmp_indexes = np.where(clustering.labels_ == label)[0]
    tmp = df_without_outliers.loc[tmp_indexes].reset_index().drop(columns=['index'])
    if(len(tmp)):
        print(f'Cluster {label}:')
        print(f'Number of elements: {len(tmp)}')
        # print(f'Labels: {len(df_without_outliers)}')
        # print(tmp[['ph', 'Hardness', 'Potability']].describe())
        print(tmp.describe())
        # print(tmp[['ph', 'Hardness', 'Potability']])
    

Cluster 0:
Number of elements: 1927
                ph     Hardness        Solids  Chloramines      Sulfate  \
count  1927.000000  1927.000000   1927.000000  1927.000000  1927.000000   
mean      7.077061   196.841051  21724.826705     7.095807   333.546763   
std       1.579917    30.113581   8453.884825     1.494075    32.833879   
min       0.000000    98.452931    320.942611     1.683993   205.935091   
25%       6.095820   178.284843  15621.461434     6.167605   313.022043   
50%       7.061831   197.226981  20857.908219     7.089146   333.084455   
75%       8.038729   215.910174  26954.499550     8.068454   352.916875   
max      14.000000   304.235912  56867.859236    12.653362   455.451234   

       Conductivity  Organic_carbon  Trihalomethanes    Turbidity  Potability  
count   1927.000000     1927.000000      1927.000000  1927.000000      1927.0  
mean     426.508979       14.353173        66.233664     3.961825         0.0  
std       77.636626        3.336756        15.76